--- Day 12: Hill Climbing Algorithm ---

You try contacting the Elves using your handheld device, but the river you're following must be too low to get a decent signal.

You ask the device for a heightmap of the surrounding area (your puzzle input). The heightmap shows the local area from above broken into a grid; the elevation of each square of the grid is given by a single lowercase letter, where a is the lowest elevation, b is the next-lowest, and so on up to the highest elevation, z.

Also included on the heightmap are marks for your current position (S) and the location that should get the best signal (E). Your current position (S) has elevation a, and the location that should get the best signal (E) has elevation z.

You'd like to reach E, but to save energy, you should do it in as few steps as possible. During each step, you can move exactly one square up, down, left, or right. To avoid needing to get out your climbing gear, the elevation of the destination square can be at most one higher than the elevation of your current square; that is, if your current elevation is m, you could step to elevation n, but not to elevation o. (This also means that the elevation of the destination square can be much lower than the elevation of your current square.)

For example:
```
Sabqponm
abcryxxl
accszExk
acctuvwj
abdefghi
```
Here, you start in the top-left corner; your goal is near the middle. You could start by moving down or right, but eventually you'll need to head toward the e at the bottom. From there, you can spiral around to the goal:
```
v..v<<<<
>v.vv<<^
.>vv>E^^
..v>>>^^
..>>>>>^
```
In the above diagram, the symbols indicate whether the path exits each square moving up (^), down (v), left (<), or right (>). The location that should get the best signal is still E, and . marks unvisited squares.

This path reaches the goal in 31 steps, the fewest possible.

What is the fewest steps required to move from your current position to the location that should get the best signal?

In [23]:
var input = System.IO.File.ReadAllLines("input.txt");
input.Take(10)

index,value
0,abccccaaaaaaaaaaaaaccaaaaaaaacccccccccaaaaaaaaccccccccaaacaaacccccccaaaaaaccccccccccccccccccccccaaaacccccccccccacccccccccccccccccccccccccccccccccccccccccccccccaaaa
1,abccccaaaaacaaaaaaccccaaaaaaccccccccccaaaaaaacccccccccaaaaaaacccccaaaaaaaaaacccccccccccccccccccaaaaaacccccccccaaaaaaaaccccccccccccccccccccccccccccccccccccccccaaaaa
2,abcccaaaaaccaaaaaaccccaaaaaaccccccaacccaaaaaacccccccccaaaaaacccaaaaaaaaaaaaaaacaaccacccccccccccaaaaaaccccccccccaaaaaacccccccccccccccccccccccccccccccccccccccccaaaaa
3,abccccccaaccaaaaaaccaaaaaaaaccccccaaacaaaacaaacccccccaaaaaaaaccaaaaaaaaaaaaaaacaaaaacccccccccccccaaccccccccccccaaaaaaccccccccccccccccccccccccccccacccccccccccaaaaaa
4,abccccccccccaaccaaccaaaaccaacccccccaaaaaaaccccccccccaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaacccccccaaaaccccccccccccccccaaaaaaaacccccccccccccccccccccccccccaaccccccccccccccaa
5,abcccccccaaaaacccaaaaaaaacccaaccccaaaaaaccccccccccccaaaaaaaaaaaaaaaacaaaaaaaccaaaaaaccccccaaaaaccccccccccccccaaaaaaaaaaccaccccccccccccccccccccccccaccccccccccccccca
6,abcccccccaaaaacccaaaaaaaaccaaaaccaaaaaaaaccccccccccccccaaacaaaaaaaaacaaaaaacccccaaaacccccaaaaaaccccccccccccccaaaaaaaaaaaaacccccccccccccccllllllccccdccccccccccccccc
7,abccccccaaaaaacccccaaaaccccaaaaacaaaaaaaaccccccccccccccaaacccccaaaccccaaaaaacccaaccccccccaaaaaacccccccccccccccccaaaaaaaaaacccccccccccccklllllllllcddddccaccaaaccccc
8,abccccccaaaaaacccccaaaaaaaaaaaaaaaccaaccccccaacaacccccccaaccccccccccccaaacaacccccccccccccaaaaaacccccccccccccccccaaaaaaaaaacccccccccccckklllppllllcddddddddaaaaccccc
9,abccccccaaaaaaccccaaacaaaaaaaaaaaaccaaccccccaaaaaccccccccccccccccccccccccccccccccccccccccccaaccccccaaccccccccccccaacaaaaaaaccccccccccckklpppppplllmdddddddddacccccc


In [24]:
public record struct Position {
    public int X;
    public int Y;

    public static Position operator+(Position position, Position other) {
        return new Position {
            X = position.X + other.X,
            Y = position.Y + other.Y,
        };
    }
    
    public static Position operator-(Position position, Position other) {
        return new Position {
            X = position.X - other.X,
            Y = position.Y - other.Y,
        };
    }
}

int ParseElevation(char c) {
    return c switch {
        'S' => 1,
        'E' => 26,
        _ => c - 'a' + 1,
    };
}

public record struct Map {
    public Dictionary<Position, int> Elevations;
    public Position Start;
    public Position End;
}

Map ParseMap(IEnumerable<string> input) {
    var elevation = new Dictionary<Position, int>();
    Position? start = null;
    Position? end  = null;
    foreach ((string line, int y) in input.Select((line, y) => (line, y))) {
        foreach ((char c, int x) in line.Select((c, x) => (c, x))) {
            elevation.Add(new Position { X = x, Y = y}, ParseElevation(c));
            if (c == 'S') {
                start = new Position { X = x, Y = y };
            }
            if (c == 'E') {
                end = new Position { X = x, Y = y };
            }
        }
    }
    return new Map {
        Elevations = elevation,
        Start = (Position) start,
        End = (Position) end,
    };
}

public record struct ExploreAction {
    public Position Position;
    public int steps;
}

public int ExploreMap(Map map) {
    var deltas = new List<Position> {
        new Position { X = 1, Y = 0 },
        new Position { X = -1, Y = 0 },
        new Position { X = 0, Y = 1 },
        new Position { X = 0, Y = -1 },
    };

    var seenPositions = new HashSet<Position>();
    var positionsToExplore = new PriorityQueue<ExploreAction, int>();
    positionsToExplore.Enqueue(new ExploreAction { Position = map.Start, steps = 0 }, 0);

    while(positionsToExplore.TryDequeue(out ExploreAction nextPositionToExplore, out int steps)) {
        var currentPosition = nextPositionToExplore.Position;
        if (currentPosition == map.End) {
            return steps;
        }
        if (seenPositions.Contains(currentPosition)) {
            continue;
        }

        var newPositions = deltas.Select(d => currentPosition + d);
        foreach (var newPos in newPositions) {
            if (map.Elevations.ContainsKey(newPos) && !seenPositions.Contains(currentPosition)) {
                if (map.Elevations[newPos] < map.Elevations[currentPosition] + 2) {
                    positionsToExplore.Enqueue(new ExploreAction { Position = newPos, steps = steps + 1 }, steps + 1);
                }
            }
        }

        seenPositions.Add(currentPosition);
    }
    throw new System.InvalidOperationException("No route found");
}

public int FindTrail(Map map) {
    var deltas = new List<Position> {
        new Position { X = 1, Y = 0 },
        new Position { X = -1, Y = 0 },
        new Position { X = 0, Y = 1 },
        new Position { X = 0, Y = -1 },
    };

    var seenPositions = new HashSet<Position>();
    var positionsToExplore = new PriorityQueue<ExploreAction, int>();
    positionsToExplore.Enqueue(new ExploreAction { Position = map.End, steps = 0 }, 0);

    while(positionsToExplore.TryDequeue(out ExploreAction nextPositionToExplore, out int steps)) {
        var currentPosition = nextPositionToExplore.Position;
        if (map.Elevations[currentPosition] == 1) {
            return steps;
        }
        if (seenPositions.Contains(currentPosition)) {
            continue;
        }

        var newPositions = deltas.Select(d => currentPosition + d);
        foreach (var newPos in newPositions) {
            if (map.Elevations.ContainsKey(newPos) && !seenPositions.Contains(currentPosition)) {
                if (map.Elevations[newPos] > map.Elevations[currentPosition] - 2) {
                    positionsToExplore.Enqueue(new ExploreAction { Position = newPos, steps = steps + 1 }, steps + 1);
                }
            }
        }

        seenPositions.Add(currentPosition);
    }
    throw new System.InvalidOperationException("No route found");
}

In [25]:
var map = ParseMap(input);
ExploreMap(map)

534

--- Part Two ---

As you walk up the hill, you suspect that the Elves will want to turn this into a hiking trail. The beginning isn't very scenic, though; perhaps you can find a better starting point.

To maximize exercise while hiking, the trail should start as low as possible: elevation a. The goal is still the square marked E. However, the trail should still be direct, taking the fewest steps to reach its goal. So, you'll need to find the shortest path from any square at elevation a to the square marked E.

Again consider the example from above:
```
Sabqponm
abcryxxl
accszExk
acctuvwj
abdefghi
```
Now, there are six choices for starting position (five marked a, plus the square marked S that counts as being at elevation a). If you start at the bottom-left square, you can reach the goal most quickly:
```
...v<<<<
...vv<<^
...v>E^^
.>v>>>^^
>^>>>>>^
```
This path reaches the goal in only 29 steps, the fewest possible.

In [26]:
var map = ParseMap(input);
FindTrail(map)

525